<a href="https://colab.research.google.com/github/talktovishal/NLP-ML-Projects/blob/master/Project2_word2vec-Part1-DownloadPreprocessKaggleData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!pip install kaggle


In [0]:
!mkdir /root/.kaggle


In [0]:
#to-do  = remove the key before saving it to github
!echo '{"username":"thevishalchowdhary","key":"XXXXXXXX"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
import json
import zipfile
import os

In [5]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
unanimad/dataisbeautiful                                    Reddit - Data is Beautiful                           11MB  2020-03-21 22:28:28            794  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)  646MB  2020-03-20 23:31:34          28735  
rubenssjr/brasilian-houses-to-rent                          brazilian_houses_to_rent                            117KB  2020-03-15 01:12:22            750  
sudalairajkumar/novel-corona-virus-2019-dataset             Novel Corona Virus 2019 Dataset                     412KB  2020-03-24 06:10:13          79121  
kimjihoo/coronavirusdataset                                 Data

In [6]:
!kaggle datasets download snapcrack/all-the-news

 89% 217M/244M [00:06<00:01, 19.1MB/s]
100% 244M/244M [00:06<00:00, 41.0MB/s]


In [7]:
!os.chdir('/content')
for file in os.listdir():
    print(file)
    if(file.endswith('.zip')):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

/bin/bash: -c: line 0: syntax error near unexpected token `'/content''
/bin/bash: -c: line 0: `os.chdir('/content')'
.config
all-the-news.zip
sample_data


In [8]:
!pwd

/content


In [0]:
!cd sample_data

In [10]:
!ls -a -l

total 903344
drwxr-xr-x 1 root root      4096 Mar 25 05:17 .
drwxr-xr-x 1 root root      4096 Mar 25 05:03 ..
-rw-r--r-- 1 root root 255356300 Mar 25 05:17 all-the-news.zip
-rw-r--r-- 1 root root 203539364 Mar 25 05:17 articles1.csv
-rw-r--r-- 1 root root 225757056 Mar 25 05:17 articles2.csv
-rw-r--r-- 1 root root 240344348 Mar 25 05:17 articles3.csv
drwxr-xr-x 1 root root      4096 Mar 23 16:11 .config
drwxr-xr-x 1 root root      4096 Mar 18 16:23 sample_data


In [11]:
import nltk
#for stop words
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string

lemmatizer = WordNetLemmatizer()

#lemmatizer code from https://simonhessner.de/lemmatize-whole-sentences-with-python-and-nltks-wordnetlemmatizer/
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:          
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:            
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return res_words


import re
# If you need to use the regex more than once it is suggested to compile it.
notStartsWithAlphaNumber = re.compile(r"[^\w\s]")
onlyDigits = re.compile(r"\d+")

def normalizeSentence(sentence):
  #lowercase
  #There's a risk of doing lower(). US and us are two different things
  updatedSentence = sentence.lower()
  #remove punctuations
  updatedSentence = updatedSentence.translate(str.maketrans('','',string.punctuation))
  updatedSentence = notStartsWithAlphaNumber.sub("", updatedSentence)
  updatedSentence = onlyDigits.sub("", updatedSentence)  
  return updatedSentence


#not i am not removing stop words since i plan to use this for tf-idf calculation
def normalizeAndLemmatizeSentence(sentence):
  updatedSentence = normalizeSentence(sentence)
  #lemmatization
  return lemmatize_sentence(updatedSentence)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
import nltk.data
nltk.download('punkt')
sentence_detection = nltk.data.load('tokenizers/punkt/english.pickle')
#https://www.nltk.org/api/nltk.tokenize.html
def extract_sentences(text):
  return sentence_detection.tokenize(text)


def extract_n_normalize_sentences(text):
  allSentences = sentence_detection.tokenize(text)
  updatedSentences = []
  for sentence in allSentences:
    #NOTE: You need an additional [] bracket before you add content to the list
    #Reason = https://stackoverflow.com/questions/29947007/array-extendstring-adds-every-character-instead-of-just-the-string
    updatedSentences.extend([normalizeSentence(sentence)])
  return updatedSentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
normalizedSentences = []
for file in os.listdir():
    if(file.endswith('.csv')):
      pdf = pd.read_csv(file)
      print(f'Read file {file} into a panda dataframe')
      
      #this is a very slow method to iterate over the list and apply the normlize function.
      #do somethign better: https://chrisalbon.com/python/basics/applying_functions_to_list_items/
      #pdf['content'].apply(lambda content: normalizedSentences.extend(extract_n_normalize_sentences(content)))

      pdf['content'].apply(lambda content: normalizedSentences.extend([normalizeSentence(s) for s in extract_sentences(content)]))
      #for testing only, read only one file...
      #break
      #pdf.info()
      #pdf.head(3)

Read file articles1.csv into a panda dataframe
Read file articles2.csv into a panda dataframe
Read file articles3.csv into a panda dataframe


In [0]:
# pdf = pd.read_csv('articles1.csv')
# print(f'Read file articles1.csv into a panda dataframe')
# pdf.info()
# pdf.head(3)


In [0]:
# pdf['content'][0]
# normalizedSentences = []
# pdf['content'].apply(lambda content: normalizedSentences.extend([content]))

In [16]:
normalizedSentences[6749]

'mr garfield said he read the script wept     mr doss had   rescued  men from a brutal battleground in japan     and that any hesitation he might have had vanished after he asked mr gibson about the controversy  '

In [0]:
import pickle

with open('normalizedSentences.pickle', 'wb') as f:
    pickle.dump(normalizedSentences, f)

In [0]:
#to make everything else run faster during testing\development.
normalizedSentences = normalizedSentences[:100000]
with open('mini-normalizedSentences.pickle', 'wb') as f:
    pickle.dump(normalizedSentences, f)

In [21]:
!pwd


/content


In [0]:
len(normalizedSentences)


50000